### Test_image_info_from_EXIF

Test snippets for reading EXIF from a directory of images.
Read all of the files in a directory and generate a .csv file with name, lat, lon, altitude

Part of this was adapted from https://www.codingforentrepreneurs.com/blog/extract-gps-exif-images-python/

In [1]:
import sys
#import glob
import os
#from os.path import isfile, join, basename
#from os import listdir
import numpy as np

# EXIF Reader
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [2]:
def latlon_from_gps_data(gps_data):
    try: 
        v = gps_data['GPSLatitude']
        lat = float(v[0][0])/float(v[0][1])+\
        (1./60.)*float(v[1][0])/float(v[1][1])+\
        (1./3600.)*float(v[2][0])/float(v[2][1])
        if gps_data['GPSLatitudeRef']=='S':
            lat = -1.*lat

        v = gps_data['GPSLongitude'] 
        lon = float(v[0][0])/float(v[0][1])+\
        (1./60.)*float(v[1][0])/float(v[1][1])+\
        (1./3600.)*float(v[2][0])/float(v[2][1])
        if gps_data['GPSLongitudeRef']=='W':
            lon = -1.*lon           
        try:
            v = gps_data['GPSAltitude']
            alt = float(v[0])/float(v[1])
        except:
            alt=np.nap
            
        return lat, lon, alt
    
    except:
        return np.nan, np.nan, np.nan

def get_gps_data(i):
    info = i._getexif()
    exif_data={}
    lat = np.nan
    lon = np.nan
    alt = np.nan
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]
                lat,lon,alt=latlon_from_gps_data(gps_data)
    else:
        print('Could not find EXIF data in ',filepath)
        
    return lat,lon,alt

In [4]:
#dn = 'C:/crs/proj/2015_Sandwich/2016-02-11_PT_images/down/'
#fn = 'DJI_0001.JPG'

path = '.'
outfile = 'gps_list.csv'
fout = open(outfile, 'w')
files = [f for f in os.listdir(path) if ( f.endswith('.jpg') or f.endswith('.JPG'))]
print(files)
print(os.path.join(path,files[0]))
for file in files:
    i = Image.open(os.path.join(path,file))
    if i:
        lat,lon,alt = get_gps_data(i)
        fout.write('{0},{1:.8f},{2:.8f},{3:.3f}\n'.format(file,lat,lon,alt))
        print('{0},{1:.8f},{2:.8f},{3:.3f}'.format(file,lat,lon,alt))

    else:
        print('Could not open ',dn+file)
        
fout.close()

['DJI_0009.JPG', 'image_no_gps.jpg', 'U0001559.JPG']
.\DJI_0009.JPG
DJI_0009.JPG,41.76494539,-70.48121569,59.951
image_no_gps.jpg,nan,nan,nan
U0001559.JPG,nan,nan,nan


In [ ]:
for tag, value in info.items():
    decoded = TAGS.get(tag, tag)
    print(decoded)

In [ ]:
files = [f for f in listdir(dn) if isfile(join(dn, f))]
print(files)